In [ ]:
# %load_ext cudf.pandas
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
from concurrent.futures import ThreadPoolExecutor
import os
    
from tqdm import tqdm
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import VotingClassifier

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import cohen_kappa_score # To calculate quadratic weighted Kappa
from sklearn.pipeline import Pipeline


from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV


from lightgbm import LGBMRegressor, LGBMClassifier

from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.base import clone, BaseEstimator, RegressorMixin

import lightgbm as lgb
from catboost import CatBoostClassifier, Pool


from scipy.stats import mode
from scipy.optimize import minimize

import torch
import torch.nn as nn
import torch.optim as optim

running_locally = True
rerun_series = False
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
if running_locally:
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')
    data_dictionary = pd.read_csv('data_dictionary.csv')
else:
    train_df = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
    test_df = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
    data_dictionary = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/data_dictionary.csv')

In [ ]:
train_df.describe()

# Exploratory Data Analysis

### Severity Impairment Index
First, we'll look at the distribution of `sii` data

In [ ]:
sii_data = train_df['sii'].value_counts().reset_index()
sii_data.columns = ['sii_label', 'count']
sii_data['percent_of_total'] = round((sii_data['count'] / len(train_df)) * 100,1)
sii_data
print(f"{sii_data}\n")
print(f"{100.0*train_df['sii'].isnull().sum()/len(train_df):.2f}% of the sii data is null")

### Demographics
Let's look through the data related to demographics

In [ ]:
train_df['Basic_Demos-Sex'].value_counts()

In [ ]:
plt.style.use('seaborn-v0_8')
sns.boxplot(x='Basic_Demos-Age', y='sii', data=train_df)
plt.title('SII by Enrollment Season')

plt.show()

In [ ]:
age_sii_counts = train_df.groupby(['Basic_Demos-Age', 'sii']).size().reset_index(name='count')
sns.barplot(data=age_sii_counts, x='Basic_Demos-Age', y='count', hue='sii')
plt.title('Counts of Basic_Demos-Age by sii Category')
plt.xlabel('Basic_Demos-Age')
plt.ylabel('Count')
plt.legend(title='sii')
plt.xticks(rotation=45)
plt.show()

Here's what we learned:
* We learned that our dataset is mostly males, about 63% male to 37% female
* Also, the interquartile range of `sii` increases with older ages. 
* Younger ages are prone to less severe impairment
* The magnitude of the severity changes based on the age in consideration. E.g.
    * 0 risk -> 8 yrs old
    * 1 risk -> 10 yrs old
    * 2 risk -> 12 yrs old
    * 3 risk -> 13 yrs old

Not shown:
* No trend between `Basic_Demos-Enroll_Season` and `sii`

### Children's Global Assessment Scale

In [ ]:
heatmap_data = train_df.groupby(['CGAS-CGAS_Score', 'CGAS-Season'])['sii'].mean().reset_index()
heatmap_data = heatmap_data.pivot(index='CGAS-CGAS_Score', columns='CGAS-Season', values='sii')
column_order = ["Spring", "Summer", "Fall", "Winter"]
heatmap_data = heatmap_data.reindex(columns=column_order) # want the seasons to be in chronological
sns.heatmap(heatmap_data, annot=True, cmap='viridis', fmt=".2f") 
plt.title('Average Problematic Internet Use (sii) by CGAS Score and Season')
plt.xlabel('CGAS Season')
plt.ylabel('CGAS Score')
plt.show()

No notable trends seen in the Children's Global Assessment Scale. Let's move on to the next instrument

### Physical Measures

In [ ]:
phys_cols = list(train_df.filter(like='Physical-').columns) + ['sii']
train_df[phys_cols].describe()

In [ ]:
sns.boxplot(x='sii', y='Physical-BMI', data=train_df)
plt.title('BMI vs. Problematic Internet Use')
plt.xlabel('Sii')
plt.ylabel('BMI (kg/m^2)')
plt.show()


In [ ]:
fig, axes = plt.subplots(1, 3)
sns.histplot(train_df[train_df['Basic_Demos-Sex'] == 0]['Physical-Height'], kde=True, ax=axes[0])
axes[0].set_title('Height Distribution (Male)')

sns.histplot(train_df[train_df['Basic_Demos-Sex'] == 1]['Physical-Height'], kde=True, ax=axes[1])
axes[1].set_title('Height Distribution (Female)')

sns.histplot(train_df['Physical-Height'], kde=True, ax=axes[2])
axes[2].set_title('Height Distribution (All)')

plt.suptitle('Height Distributions')
plt.xlabel('Height (inches)')
plt.show()

Here's what we learned: 
- Higher BMI corresponds to higher `sii`. The 50 percentile of BMI for "Severe" sii is higher than the 75 percentile for "Moderate" sii
- There was minimal variance in the season w.r.t. the `sii`
- The distribution of heights seemed to take on the form of a bimodal distribution instead of a typical normal distribution. This is likely because males and females are different heights, so the resulting histogram has 2 peaks. The males are the shorter peak because they are typically shorter in the age range collected in this dataset.

### FitnessGram Vitals and Treadmill

In [ ]:
fitness_endurance_cols = list(train_df.filter(like='Fitness_Endurance-').columns) + ['sii']
train_df[fitness_endurance_cols].describe()

In [ ]:
# Hardcode the columns to avoid the category columns
train_df[['Fitness_Endurance-Max_Stage','Fitness_Endurance-Time_Mins','Fitness_Endurance-Time_Sec','sii']].corr()

In [ ]:
sns.histplot(train_df, x='Fitness_Endurance-Time_Mins',bins=20, hue='sii')
plt.title('Fitness Endurance Time vs. Problematic Internet Use')
plt.xlabel('Count')
plt.ylabel('Fitness_Endurance-Time_Mins')
plt.show()

Here's what we learned:
- `Fitness_Endurance-Time_Mins` and `Fitness_Endurance-Time_Sec` should be combined into 1 column. This will be done later
- There wasnt a strong correlation with any of these columns and `sii`

### FitnessGram Child

In [ ]:
fgc_cols = list(train_df.filter(like='FGC').columns) + ['sii']
train_df[fgc_cols].describe()

In [ ]:
corr_matrix = train_df[fgc_cols].drop(columns=['FGC-Season']).corr()
sorted_cols = corr_matrix.columns.sort_values()
sorted_corr_matrix = corr_matrix.loc[sorted_cols, sorted_cols]

sns.heatmap(sorted_corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

Here's what we learned:
- Interestingly, there isn't as high of a correlation between the raw values and the categorized "zones". 
    - The highest observed correlation between a category and its zone was only 0.75 for `FGC_FGC_SRR` and `FGC_FGC_SRL`. (These two measurements are just the left and right legs of the Sit & Reach.)

### Bio-electric Impedance Analysis

In [ ]:
bia_cols = list(train_df.filter(like='BIA-').columns) + ['sii']
train_df[bia_cols].describe()

In [ ]:
bia_corr_matrix = train_df[bia_cols].drop(columns=['BIA-Season']).corr()
bia_sorted_cols = bia_corr_matrix.columns.sort_values()
bia_sorted_corr_matrix = bia_corr_matrix.loc[bia_sorted_cols, bia_sorted_cols]

sns.heatmap(bia_sorted_corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

We can clearly see that there are some highly correlated features. Let's explore that a bit more

In [ ]:
pd.set_option('display.max_rows', None)
correlated_bia_features = ['BIA-BIA_BMR','BIA-BIA_DEE','BIA-BIA_ECW','BIA-BIA_FFM','BIA-BIA_Fat','BIA-BIA_ICW','BIA-BIA_LDM','BIA-BIA_LST','BIA-BIA_SMM','BIA-BIA_TBW']

In [ ]:
# TODO: uncomment this later
# pairplot_features = correlated_bia_features + ['sii']
# sns.pairplot(train_df[pairplot_features], hue='sii', palette='viridis')

Here's what we learned:
- Body Fat Percentage is inversely correlated to most BIA fields, i.e. Bone Mineral Content, Basal Metabolic Rate, Daily Energy Expenditure, Extracellular Water, Fat Free Mass, Intracellular Water, Lean Dry Mass, Lean Soft Tissue, Skeletal Muscle Mass, Total Body Water.
- Basal Metabolic Rate, Daily Energy Expenditure, Extracellular Water, and Fat Free Mass are all correlated to each other.
- Similarly, Intracellular Water, Lean Dry Mass, Lean Soft Tissue, Skeletal Muscle Mass, and Total Body Water are correlated to each other
- No one attribute correlates to `sii`

### Physical Activity Questionnaire

In [ ]:
fig, axes = plt.subplots(1,2)
sns.barplot(data=train_df, x='PAQ_A-Season', y='PAQ_A-PAQ_A_Total', hue='sii', ax=axes[0])
axes[0].set_title('Physical Activity Questionnaire (Adults)')

sns.barplot(data=train_df, x='PAQ_C-Season', y='PAQ_C-PAQ_C_Total', hue='sii', ax=axes[1])
axes[1].set_title('Physical Activity Questionnaire (Children)')

Here's what we learned:
- There are no adults with a "severe" `sii` rating in the Summer.
- Winter was the season with the most "severe" `sii` ratings for children, but it was the season with the fewest "severe" `sii` ratings for adults. 

### Parent-Child Internet Addiction Test

In [ ]:
pciat_aggs = train_df.groupby('sii')['PCIAT-PCIAT_Total'].agg(['min', 'max', 'mean'])
pciat_aggs = pciat_aggs.rename(
    columns={'min': 'Minimum PCIAT total Score', 'max': 'Maximum total PCIAT Score', 'mean': 'Average total PCIAT Score'}
)
pciat_aggs

In [ ]:
# List all columns that are of the form PCIAT-PCIAT_XX
pciat_columns = [f'PCIAT-PCIAT_{i:02d}' for i in range(1, 21)]

pciat_summed_total = train_df[pciat_columns].fillna(0).sum(axis=1)
is_calculated_sum_equal_to_total_column = pciat_summed_total == train_df['PCIAT-PCIAT_Total'].fillna(0)

# Check if sum of the scores equals total for all rows
is_calculated_sum_equal_to_total_column.sum() == len(train_df)


Here's what we learned:
- The average of the scores for each `sii` classification was around the median of the bounds of the range (e.g. n `sii` of 0 is from 0-30, and the average score was ~14)
- The sum of the PCIAT scores all align with the classification set by the Severity Impairment Index (sii).
- The number of instances where PCIAT-PCIAT_1 through PCIAT-PCIAT_20 is equal to PCIAT-PCIAT_Total is equal to the number of records in the dataset. This shows that PCIAT-PCIAT_Total is 100% is a linear transformation of other features in this dataset, so we can drop it from the dataset later. However, many of the columns are `nan`. This means that if an entry's sum is 30 but, for instance, there are 10 `nan` values, that record is likely misclassified. We will account for this in the Feature Engineering section.

Also, note that the **test** set does not have any of the PCIAT data, and we're missing about 30% of the `sii` label in the training data. 

### Sleep Disturbance Scale

In [ ]:
sds_cols = list(train_df.filter(like='SDS-').columns) + ['sii']
train_df[sds_cols].drop(columns=['sii']).isnull().sum()/train_df[['SDS-Season','SDS-SDS_Total_Raw','SDS-SDS_Total_T']].notnull().sum()

In [ ]:
# Hardcode to omit the category columns
train_df[['SDS-SDS_Total_Raw','SDS-SDS_Total_T']].corr()

Here's what we learned:
- About half of the sleep data is null
- There is an extremely strong correlation between the Total Raw Score and Total T-score. We will remove one in the Feature Engineering Section

### Internet Use

In [ ]:
int_use = list(train_df.filter(like='PreInt_EduHx-').columns) + ['sii']
train_df[int_use].drop(columns='PreInt_EduHx-Season').corr()

Here's what we learned:
- There is a weak correlation between Hours of Using Computer/Internet and `sii`. This is the strongest correlation we have seen so far between a raw column and `sii`.

# Series Data

In [ ]:
if rerun_series:
    def process_file(filename, dirname):
        data = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
        data.drop('step', axis=1, inplace=True)
        return data.describe().values.reshape(-1), filename.split('=')[1]

    def load_time_series(dirname) -> pd.DataFrame:
        ids = [d for d in os.listdir(dirname) if os.path.isdir(os.path.join(dirname, d))]

        with ThreadPoolExecutor() as executor:
            results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
        stats, indexes = zip(*results)

        data = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
        data['id'] = indexes
        return data

    if running_locally:
        train_ts = load_time_series('./series_train.parquet')
        test_ts = load_time_series('./series_test.parquet')
        random_raw_parquet_file = pd.read_parquet('series_train.parquet/id=0417c91e/part-0.parquet')
    else:
        train_ts = load_time_series('/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet')
        test_ts = load_time_series('/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet')

    train_ts.to_csv('train_ts.csv', index=False)
    test_ts.to_csv('test_ts.csv', index=False)


In [ ]:
if not rerun_series:
    train_ts = pd.read_csv('train_ts.csv')
    test_ts = pd.read_csv('test_ts.csv')

# Data Cleaning and Feature Engineering

Combine `Fitness_Endurance-Time_Mins` and `Fitness_Endurance-Time_Sec` into 1 column

In [ ]:
def combine_fe_time_into_one_column(df):
    # check if either column is null
    null_mask = df['Fitness_Endurance-Time_Mins'].isnull() | df['Fitness_Endurance-Time_Sec'].isnull()
    df['Fitness_Endurance-Time'] = df['Fitness_Endurance-Time_Mins'] + df['Fitness_Endurance-Time_Sec'] / 60

    # set result to null if either column is null
    df.loc[null_mask, 'Fitness_Endurance-Time'] = np.nan  

    df.drop(columns=['Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec'], inplace=True)

combine_fe_time_into_one_column(test_df)
combine_fe_time_into_one_column(train_df)

`SDS-SDS_Total_Raw` and `SDS-SDS_Total_T` have a 0.996 correlation. Remove the one smaller effect on the target variable. We can use a Random Forest for this.

In [ ]:
model = RandomForestClassifier(random_state=42)
temp_df = train_df[train_df['sii'].notnull()].copy()
temp_df.dropna(subset=['SDS-SDS_Total_Raw', 'SDS-SDS_Total_T'], inplace=True)
model.fit(temp_df[['SDS-SDS_Total_Raw', 'SDS-SDS_Total_T']], temp_df['sii'])

print(*zip(model.feature_importances_, model.feature_names_in_))

`SDS-SDS_Total_T` has a smaller effect on the target variable, so we will drop it

In [ ]:
def remove_sds_column(df):
    if 'SDS-SDS_Total_T' in df.columns:
        df.drop(columns=['SDS-SDS_Total_T'], inplace=True)
remove_sds_column(train_df)
remove_sds_column(test_df)

### PCIAT
There is a data point with no PCIAT scores but a total value of 0. It has an `PreInt_EduHx-computerinternet_hoursday` of 2, but an `sii` of 0. If over 70% of the PCIAT scores are null, then we cannot rely on the `PCIAT-PCIAT_Total` score, so we null it out.

In [ ]:
pciat_component_cols = [f'PCIAT-PCIAT_{i:02}' for i in range(1, 21)]
null_percentage = train_df[pciat_component_cols].isnull().mean(axis=1)

mask = null_percentage > 0.7  # Can adjust this to make it more or less strict
train_df.loc[mask, 'PCIAT-PCIAT_Total'] = np.nan
# No PCIAT columns on test_df, so don't need to transform it

If a record is missing multiple values in the PCIAT evaluations, it could potentially be missing data that would **increase the severity** of the `sii`. The dataset treats those values as 0, but that is not a fair assumption. If there are a sufficient values from a row missing, we should impute the missing values using the average value for that column and re-calculate the `PCIAT-PCIAT_TOTAL` (and subsequently, the `sii`).

In [ ]:
# Only operate on rows where the score is already >= 10. With only 20 columns, getting to 30 points to move up in severity won't happen.
temp_pciat_df = train_df[(train_df['PCIAT-PCIAT_Total'] >= 10) & (train_df[pciat_component_cols].isnull().any(axis=1))].copy()

temp_pciat_df_filled = temp_pciat_df[pciat_component_cols].apply(lambda x: x.fillna(x.mean()), axis=0)
temp_pciat_df['PCIAT_Sum_Filled'] = temp_pciat_df_filled.sum(axis=1)

train_df.loc[temp_pciat_df.index, 'PCIAT_Sum_Filled'] = temp_pciat_df['PCIAT_Sum_Filled']

In [ ]:
def assign_bucket(value):
    if 0 <= value <= 30:
        return 0
    elif 31 <= value <= 49:
        return 1
    elif 50 <= value <= 79:
        return 2
    elif 80 <= value:
        return 3
    else:
        return np.nan

# Apply the bucket assignment function to both columns
train_df['PCIAT_Sum_Classify_Sii'] = train_df['PCIAT_Sum_Filled'].apply(assign_bucket)

rows_to_be_updated = train_df[train_df['PCIAT_Sum_Classify_Sii'] > train_df['sii']]
print(rows_to_be_updated[['sii', 'PCIAT_Sum_Classify_Sii']])

train_df['sii'] = np.where(
    train_df['PCIAT_Sum_Classify_Sii'] > train_df['sii'],  # Condition
    train_df['PCIAT_Sum_Classify_Sii'],                    # New value if condition is met
    train_df['sii']                                        # Keep existing value otherwise
)

train_df.drop(columns=['PCIAT_Sum_Filled', 'PCIAT_Sum_Classify_Sii'], inplace=True)

We should also check for outliers and replace them with `NaN`. As an example, `CGAS-CGAS_Score` has a value of 999, which is an error

In [ ]:
def handle_outliers(df, column, valid_min, valid_max, placeholder_value=np.nan):
    outliers = (df[column] < valid_min) | (df[column] > valid_max)
    
    print(f"Found {outliers.sum()} outliers in column '{column}'.")
    
    df.loc[outliers, column] = placeholder_value

handle_outliers(train_df, 'CGAS-CGAS_Score', 0, 100, np.nan)
handle_outliers(test_df, 'CGAS-CGAS_Score', 0, 100, np.nan)
handle_outliers(train_df, 'SDS-SDS_Total_Raw', 0, 100, np.nan)
handle_outliers(test_df, 'SDS-SDS_Total_Raw', 0, 100, np.nan)

# Impute Missing Data

### Remove Sparse Records

There are a handful of records that are almost entirely null. We will remove these from the training data. However, every record has a value for`id`, `Basic_Demos-Age`, and `Basic_Demos-Sex`. Therefore, if a record has all remaining columns listes as `NaN`, it is safe to drop them from `train_df`. 

In [ ]:
pd.set_option('display.max_rows', 10)
print(f"There are {len(train_df.columns)} columns in the training data and {len(test_df.columns)} columns in the test data.\n")
null_counts_per_row = train_df.isnull().sum(axis=1)
null_counts_distribution = null_counts_per_row.value_counts().sort_index()
print(f"Here are the number of null columns each record has:\n {null_counts_distribution}")

In [ ]:
def drop_mostly_null_rows(df):
    columns_to_check = [col for col in df.columns if 'stat' not in col]
    threshold = 0.10  # require at least 10% of the dataset be non-null return df
    row_threshold = int(threshold * (len(columns_to_check)))
    df.dropna(thresh=row_threshold, subset=columns_to_check, axis=0, inplace=True)
    return df

train_df = drop_mostly_null_rows(train_df)
test_df = drop_mostly_null_rows(test_df)

### Define Feature Interaction Columns

In [ ]:
# Can try this. Tests all the combos. Beware the curse of dimensionality
# from itertools import combinations

# columns = correlated_bia_features
# feature_pairs = list(combinations(columns, 2))
# def create_interaction_features(df):
#     new_features = []
#     for f1, f2 in feature_pairs:
#         new_df = pd.DataFrame()
#         new_df[f'{f1}_x_{f2}'] = df[f1] * df[f2]
#         new_df[f'{f1}_div_{f2}'] = df[f1] / (df[f2] + 1e-5)
#         new_features.append(new_df)

#     df = pd.concat([df] + new_features, axis=1)
#     return df

# # Apply the function to both train_df and test_df
# train_df = create_interaction_features(train_df)
# test_df = create_interaction_features(test_df)


def feature_engineering(df):
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['BFP_BMI'] = df['BIA-BIA_Fat'] / df['BIA-BIA_BMI']
    df['FFMI_BFP'] = df['BIA-BIA_FFMI'] / df['BIA-BIA_Fat']
    df['FMI_BFP'] = df['BIA-BIA_FMI'] / df['BIA-BIA_Fat']
    df['LST_TBW'] = df['BIA-BIA_LST'] / df['BIA-BIA_TBW']
    df['BFP_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
    df['BFP_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']
    # df['BMR_Weight'] = df['BIA-BIA_BMR'] / df['Physical-Weight']
    # df['DEE_Weight'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
    df['SMM_Height'] = df['BIA-BIA_SMM'] / df['Physical-Height']
    df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / df['BIA-BIA_FMI']
    # df['Hydration_Status'] = df['BIA-BIA_TBW'] / df['Physical-Weight']
    df['ICW_TBW'] = df['BIA-BIA_ICW'] / df['BIA-BIA_TBW']
    df['BMI_PHR'] = df['Physical-BMI'] * df['Physical-HeartRate']
    return df

train_df = feature_engineering(train_df)
test_df = feature_engineering(test_df)

In [ ]:
train_df.describe()

### Use k-NN to Impute Missing Numeric, Non-categorical Data

In [ ]:
def get_features_numeric_and_non_categorical_features_from_df(df):
    numeric_columns = df.select_dtypes(include=['int32', 'int64', 'float64']).columns
    dd_fields = data_dictionary[data_dictionary['Type'] == 'categorical int']['Field'].tolist()

    # Only normalize numeric columns that are NOT categorical int
    features = [col for col in numeric_columns if col not in dd_fields and col != 'sii']
    return features
# features_to_normalize = get_features_numeric_and_non_categorical_features_from_df(train_df)

In [ ]:
def knn_impute(df, n_neighbors=5):
    features_to_impute = get_features_numeric_and_non_categorical_features_from_df(df)
    
    imputer = KNNImputer(n_neighbors=n_neighbors)

    # Perform k-NN imputation and ensure the result integrates with the DataFrame
    imputed_data = imputer.fit_transform(df[features_to_impute])
    df[features_to_impute] = pd.DataFrame(imputed_data, columns=features_to_impute, index=df.index)
    
    return df

train_df = knn_impute(train_df, n_neighbors=5)
test_df = knn_impute(test_df, n_neighbors=5)


### Use k-NN to Impute Missing Numeric, Categorical Data

In [ ]:
# Basic_Demos-Sex was removed from this list
numeric_categorical_data = ['FGC-FGC_CU_Zone','FGC-FGC_GSND_Zone','FGC-FGC_GSD_Zone','FGC-FGC_PU_Zone','FGC-FGC_SRL_Zone','FGC-FGC_SRR_Zone','FGC-FGC_TL_Zone','BIA-BIA_Activity_Level_num','BIA-BIA_Frame_num','PCIAT-PCIAT_01','PCIAT-PCIAT_02','PCIAT-PCIAT_03','PCIAT-PCIAT_04','PCIAT-PCIAT_05','PCIAT-PCIAT_06','PCIAT-PCIAT_07','PCIAT-PCIAT_08','PCIAT-PCIAT_09','PCIAT-PCIAT_10','PCIAT-PCIAT_11','PCIAT-PCIAT_12','PCIAT-PCIAT_13','PCIAT-PCIAT_14','PCIAT-PCIAT_15','PCIAT-PCIAT_16','PCIAT-PCIAT_17','PCIAT-PCIAT_18','PCIAT-PCIAT_19','PCIAT-PCIAT_20','PreInt_EduHx-computerinternet_hoursday']

def knn_impute_categorical(df, categorical_columns, n_neighbors=5):
    imputer = KNNImputer(n_neighbors=n_neighbors)

    # Need to check this because test_df doesn't have all the columns as train_df
    valid_columns = [col for col in categorical_columns if col in df.columns]

    # Fit-transform on valid columns
    imputed_data = imputer.fit_transform(df[valid_columns])
    df[valid_columns] = pd.DataFrame(imputed_data, columns=valid_columns, index=df.index)

    df[valid_columns] = df[valid_columns].round().astype(int)
    return df

train_df = knn_impute_categorical(train_df, numeric_categorical_data)
test_df = knn_impute_categorical(test_df, numeric_categorical_data)

# Actigraphy Feature Engineering

In [ ]:
# Was getting a kernel error for a bit. Thought you could set the device to 'mps' on Mac, but it didn't work. Set to 'cpu' and it worked, so I left it
mps_device = torch.device("mps")

class AutoEncoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim*3),
            nn.ReLU(),
            nn.Linear(encoding_dim*3, encoding_dim*2),
            nn.ReLU(),
            nn.Linear(encoding_dim*2, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, input_dim*2),
            nn.ReLU(),
            nn.Linear(input_dim*2, input_dim*3),
            nn.ReLU(),
            nn.Linear(input_dim*3, input_dim),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

def perform_autoencoder(df, encoding_dim=50, epochs=50, batch_size=32):

    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    
    data_tensor = torch.FloatTensor(df_scaled, device='cpu')
    
    input_dim = data_tensor.shape[1]
    autoencoder = AutoEncoder(input_dim, encoding_dim).to('cpu')
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters())
    
    for epoch in range(epochs):
        for i in range(0, len(data_tensor), batch_size):
            batch = data_tensor[i : i + batch_size]
            optimizer.zero_grad()
            reconstructed = autoencoder(batch)
            loss = criterion(reconstructed, batch)
            loss.backward()
            optimizer.step()
            
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}]')
                 
    with torch.no_grad():
        encoded_data = autoencoder.encoder(data_tensor).numpy()
        
    df_encoded = pd.DataFrame(encoded_data, columns=[f'Enc_{i + 1}' for i in range(encoded_data.shape[1])])
    
    return df_encoded


In [ ]:
train_ts_no_id = train_ts.drop('id', axis=1)
test_ts_no_id = test_ts.drop('id', axis=1)

if False or not running_locally:
    train_ts_encoded = perform_autoencoder(train_ts_no_id, encoding_dim=50, epochs=100, batch_size=32)
    test_ts_encoded = perform_autoencoder(test_ts_no_id, encoding_dim=50, epochs=100, batch_size=32)

    train_ts_encoded.to_csv('train_ts_encoded.csv', index=False)
    test_ts_encoded.to_csv('test_ts_encoded.csv', index=False)

else:
    train_ts_encoded = pd.read_csv('train_ts_encoded.csv')
    test_ts_encoded = pd.read_csv('test_ts_encoded.csv')

train_ts_encoded["id"] = train_ts["id"]
test_ts_encoded['id'] = test_ts["id"]

train_df = pd.merge(train_df, train_ts_encoded, how="left", on='id')
test_df = pd.merge(test_df, test_ts_encoded, how="left", on='id')



# Create the Model

First, we will define quadratic weighted kappa as our error function and create our train/test split.

In [ ]:
def calculate_qwk(y1, y2):
    return cohen_kappa_score(y1, y2, weights='quadratic')

In [ ]:
# Can't use imputation to predict the value for `sii`, so we drop those that are null
train_df.dropna(subset=['sii'], inplace=True)

X = train_df.drop(columns=['sii','id'])
y = train_df['sii']
# Need to drop all the PCIAT-PCIAT_# columns because they are not given in the test data
X = X.drop(columns=[col for col in X.columns if 'PCIAT' in col])

test_df_ids = test_df['id']
test_df = test_df.drop(columns=['id'])

Define one-hot encoding of seasonal data

In [ ]:
season_columns = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
          'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
          'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']
def one_hot_encode_seasons(train_df, test_df):    
    for col in season_columns:
        
        if col in train_df.columns and col in test_df.columns:
            all_categories = sorted(pd.concat([train_df[col], test_df[col]], axis=0).dropna().unique())
            
            train_dummies = pd.get_dummies(train_df[col], prefix=col).reindex(columns=[f"{col}_{cat}" for cat in all_categories], fill_value=0)
            test_dummies = pd.get_dummies(test_df[col], prefix=col).reindex(columns=[f"{col}_{cat}" for cat in all_categories], fill_value=0)

            train_df = pd.concat([train_df, train_dummies], axis=1)
            test_df = pd.concat([test_df, test_dummies], axis=1)

            # Drop the original column
            train_df.drop(col, axis=1, inplace=True)
            test_df.drop(col, axis=1, inplace=True)
    return train_df, test_df

def categorical_encode_seasons(train_df, test_df):
    for col in season_columns:
        if col in train_df.columns and col in test_df.columns:
            train_df[col] = train_df[col].fillna('Missing')
            test_df[col] = test_df[col].fillna('Missing')
            train_df[col] = train_df[col].astype('category')
            test_df[col] = test_df[col].astype('category')

            le = LabelEncoder() # use same encoder for both train and test
            train_df[col] = le.fit_transform(train_df[col])
            test_df[col] = le.transform(test_df[col])

    return train_df, test_df


In [ ]:
def threshold_rounder(oof_non_rounded, thresholds):
    """
    Rounds predictions to the nearest class based on thresholds.
    """
    output = np.zeros_like(oof_non_rounded, dtype=int)
    for i, threshold in enumerate(thresholds):
        output[oof_non_rounded >= threshold] = i + 1
    return output

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    """
    Evaluates predictions using quadratic weighted kappa.
    """
    rounded_preds = threshold_rounder(oof_non_rounded, thresholds)
    return -calculate_qwk(y_true, rounded_preds)

### Ensemble method

In [ ]:
null_values = X.isna()

# Find columns with any null values
columns_with_nulls = null_values.any()

print("Columns with null values:")
print(columns_with_nulls[columns_with_nulls])


In [ ]:
n_splits = 5

# Don't want to one-hot encode the original datasets, so we make a copy
X_reg, test_df_reg = categorical_encode_seasons(X, test_df)

SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

models = [
    # ("rf", RandomForestRegressor(n_estimators=200, max_depth=5, min_samples_split=2, random_state=42)),
    ("xgb", XGBRegressor(n_estimators=200, max_depth=3, learning_rate=0.05, subsample=0.8, colsample_bytree=1, reg_alpha=0.5, reg_lambda=1, random_state=42)),
    # ("gb", GradientBoostingRegressor(n_estimators=200, random_state=42)),
    ("lgb", LGBMRegressor(num_leaves=33, subsample=0.6, reg_lambda=0.5, reg_alpha=0.5, min_child_samples=10, max_depth=-1, learning_rate=0.1, colsample_bytree=0.8, random_state=42)),   
]

train_S = []
test_S = []

oof_non_rounded = np.zeros(len(y), dtype=float)
test_preds_per_model = np.zeros((len(test_df_reg), len(models), n_splits))

for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X_reg, y), desc="Training Fold", total=n_splits)):
    X_train, X_val = X_reg.iloc[train_idx], X_reg.iloc[test_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]
    
    fold_val_preds = np.zeros((len(y_val), len(models)))

    for model_idx, (name, model) in enumerate(models):
        print(f"Training model: {name} on Fold {fold + 1}")
        model.fit(X_train, y_train)

        fold_val_preds[:, model_idx] = model.predict(X_val)

        # Test predictions for this model
        test_preds_per_model[:, model_idx, fold] = model.predict(test_df_reg)
    
    # Average predictions across models for validation
    y_val_pred_ensemble = fold_val_preds.mean(axis=1)
    oof_non_rounded[test_idx] = y_val_pred_ensemble

    # Calculate metrics for each fold
    train_kappa = calculate_qwk(y_train, model.predict(X_train).round(0).astype(int))
    val_kappa = calculate_qwk(y_val, y_val_pred_ensemble.round(0).astype(int))


    train_S.append(train_kappa)
    test_S.append(val_kappa)
    print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")

final_test_preds = test_preds_per_model.mean(axis=2).mean(axis=1)


initial_thresholds = np.quantile(oof_non_rounded, [0.25, 0.5, 0.75])
optimizer = minimize(evaluate_predictions, x0=initial_thresholds, args=(y, oof_non_rounded), method='Nelder-Mead')
thresholds = optimizer.x if optimizer.success else [0.5, 1.5, 2.5]


print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")
# print("Final Test Predictions (Averaged):")
# print(final_test_preds)

# Best scores: 
# model = RandomForestRegressor(n_estimators=200, max_depth=10, min_samples_split=2, random_state=42)
#   Mean Train QWK --> 0.8003 | Mean Validation QWK ---> 0.3587


Training Fold:   0%|          | 0/5 [00:00<?, ?it/s]

Training model: lgb on Fold 1
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001976 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18237
[LightGBM] [Info] Number of data points in the train set: 2188, number of used features: 119
[LightGBM] [Info] Start training from score 0.582267


Training Fold:  20%|██        | 1/5 [00:01<00:07,  1.96s/it]

Fold 1 - Train QWK: 0.9996, Validation QWK: 0.3924
Training model: lgb on Fold 2
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002181 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18357
[LightGBM] [Info] Number of data points in the train set: 2189, number of used features: 119
[LightGBM] [Info] Start training from score 0.582915


Training Fold:  40%|████      | 2/5 [00:03<00:05,  1.96s/it]

...

Fold 3 - Train QWK: 0.9992, Validation QWK: 0.4413
Training model: lgb on Fold 4
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001574 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18184
[LightGBM] [Info] Number of data points in the train set: 2189, number of used features: 119
[LightGBM] [Info] Start training from score 0.582001


Training Fold:  80%|████████  | 4/5 [00:07<00:01,  1.93s/it]

Fold 4 - Train QWK: 0.9992, Validation QWK: 0.3685
Training model: lgb on Fold 5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001666 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18482
[LightGBM] [Info] Number of data points in the train set: 2189, number of used features: 119
[LightGBM] [Info] Start training from score 0.582001


Training Fold: 100%|██████████| 5/5 [00:09<00:00,  1.95s/it]

Fold 5 - Train QWK: 0.9992, Validation QWK: 0.4126
Mean Train QWK --> 0.9993
Mean Validation QWK ---> 0.4015


In [ ]:
oof_tuned = threshold_rounder(oof_non_rounded, thresholds)
test_predictions = threshold_rounder(final_test_preds, thresholds)

submission = pd.DataFrame({
    'id': test_df_ids.values,
    'sii': test_predictions
})
submission.to_csv('submission.csv', index=False)


In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from lightgbm import LGBMRegressor
# import numpy as np

# # Define the base model
# lgb_model = LGBMRegressor(n_estimators=200, random_state=42)

# # Define the parameter distribution
# param_distributions = {
#     'num_leaves': [20,33,45],
#     'max_depth': [-1, 10, 20],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'min_child_samples': [10,20,30],
#     'subsample': [0.6, 0.8, 1.0],
#     'colsample_bytree': [0.6, 0.8, 1.0],
#     'reg_alpha': np.linspace(0, 1.0, 5),
#     'reg_lambda': np.linspace(0, 2.0, 5)
# }

# # Create the RandomizedSearchCV object
# random_search = RandomizedSearchCV(
#     estimator=lgb_model,
#     param_distributions=param_distributions,
#     n_iter=60,  # Number of parameter settings to sample
#     scoring='neg_mean_squared_error',
#     cv=5,
#     verbose=1,
#     random_state=42,
#     n_jobs=-1
# )

# # Fit the randomized search
# random_search.fit(X_reg, y)

# # Print the best parameters and score
# print("Best Parameters:", random_search.best_params_)
# print("Best Score:", -random_search.best_score_)


In [ ]:
# print("Best Parameters:", random_search.best_params_)
# print("Best Score:", -random_search.best_score_)

# results = pd.DataFrame(random_search.cv_results_)

# results['mean_test_score'] = -results['mean_test_score']
# top_results = results.sort_values(by='mean_test_score', ascending=False).head(5)
# top_results[['mean_test_score', 'params']].to_csv('asdfasdf.csv')


### Random Forest Regressor

In [ ]:
# n_splits = 5

# # Don't want to one-hot encode the original datasets, so we make a copy
# X_reg, test_df_reg = categorical_encode_seasons(X, test_df)

# SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# test_preds = np.zeros((len(test_df_reg), n_splits))
# train_S = []
# test_S = []

# oof_non_rounded = np.zeros(len(y), dtype=float)

# for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X_reg, y), desc="Training Fold", total=n_splits)):
#     X_train, X_val = X_reg.iloc[train_idx], X_reg.iloc[test_idx]
#     y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]
    
#     model = RandomForestRegressor(n_estimators=200, max_depth=10, min_samples_split=2, random_state=42)
#     model.fit(X_train, y_train)

#     y_train_pred = model.predict(X_train)
#     y_val_pred = model.predict(X_val)

#     oof_non_rounded[test_idx] = y_val_pred

#     train_kappa = calculate_qwk(y_train, y_train_pred.round(0).astype(int))
#     val_kappa = calculate_qwk(y_val, y_val_pred.round(0).astype(int))
    
#     train_S.append(train_kappa)
#     test_S.append(val_kappa)

#     test_preds[:, fold] = model.predict(test_df_reg)
#     print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")

# print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
# print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

# initial_thresholds = np.quantile(oof_non_rounded, [0.25, 0.5, 0.75])
# optimizer = minimize(evaluate_predictions, x0=initial_thresholds, args=(y, oof_non_rounded), method='Nelder-Mead')
# thresholds = optimizer.x if optimizer.success else [0.5, 1.5, 2.5]

# # Best scores: 
# # model = RandomForestRegressor(n_estimators=200, max_depth=10, min_samples_split=2, random_state=42)
# #   Mean Train QWK --> 0.8003 | Mean Validation QWK ---> 0.3587


In [ ]:
# oof_tuned = threshold_rounder(oof_non_rounded, thresholds)
# test_predictions = threshold_rounder(test_preds.mean(axis=1), thresholds)

# submission = pd.DataFrame({
#     'id': test_df_ids.values,
#     'sii': test_predictions
# })
# submission.to_csv('submission.csv', index=False)


In [ ]:
# # TODO: Use this in the training loop
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import make_scorer, cohen_kappa_score

# def qwk_scorer(y_true, y_pred):
#     rounded_y_pred = threshold_rounder(y_pred, thresholds=[0.5, 1.5, 2.5])  # Adjust thresholds as needed
#     return cohen_kappa_score(y_true, rounded_y_pred, weights='quadratic')

# qwk_scorer = make_scorer(qwk_scorer, greater_is_better=True)  # Create a scorer object


# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [5, 10, 15],
#     'min_samples_split': [2, 5, 10]
# }

# grid_search = GridSearchCV(
#     estimator=RandomForestRegressor(random_state=42),
#     param_grid=param_grid,
#     scoring=qwk_scorer,  # Use your desired scoring metric
#     cv=5  # Use 5-fold cross-validation
# )

# grid_search.fit(X, y)

# best_model = grid_search.best_estimator_


In [ ]:
# best_score = grid_search.best_score_

# # Get the cross-validation results
# cv_results = grid_search.cv_results_

# # Find the indices of the top 2 models
# top_4_indices = np.argsort(cv_results['mean_test_score'])[-4:]  # Sort by mean test score and get the last two indices

# # Get the scores and parameters of the top 2 models
# for index in top_4_indices:
#     score = cv_results['mean_test_score'][index]
#     params = cv_results['params'][index]
#     print(f"Score: {score:.4f}, Params: {params}")


In [ ]:

# from catboost import CatBoostRegressor
# from tqdm import tqdm

# SEED = 42
# n_splits=5
# train = X.copy()
# # test = y.copy()

# # season_cols = [col for col in train.columns if 'Season' in col]
# # train = train.drop(season_cols, axis=1) 

# cat_c = ['Basic_Demos-Enroll_Season','CGAS-Season','Physical-Season','Fitness_Endurance-Season']
# # FGC-Season: object, BIA-Season: object, PAQ_A-Season: object, PAQ_C-Season: object, PCIAT-Season: object, SDS-Season: object, PreInt_EduHx-Season: object

# def update(df):
#     global cat_c
#     for c in cat_c: 
#         df[c] = df[c].fillna('Missing')
#         df[c] = df[c].astype('category')
#     return df
        
# train = update(train)
# # test = update(test)

# def create_mapping(column, dataset):
#     unique_values = dataset[column].unique()
#     return {value: idx for idx, value in enumerate(unique_values)}

# for col in cat_c:
#     mapping = create_mapping(col, train)
#     # mappingTe = create_mapping(col, test)
    
#     train[col] = train[col].replace(mapping).astype(int)
#     # test[col] = test[col].replace(mappingTe).astype(int)

# # for i in train.columns:
# #     if 'Season' in i:
# #         print(i)
# # # train['Basic_Demos-Enroll_Season']

# Old work


In [ ]:
# train_data = lgb.Dataset(X_full_train, label=y_full_train)

# # Set parameters for multi-class classification
# params = {
#     'objective': 'multiclass',
#     'metric': 'multi_logloss',
#     'num_class': 4,  # Number of classes
#     'boosting_type': 'gbdt',
#     'num_leaves': 31,
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
# }

# # Train the model
# model = lgb.train(params, train_data, num_boost_round=100)

# # Predict class probabilities for each class
# y_pred_proba = model.predict(X_test)

# # Convert probabilities to class labels (e.g., by choosing the class with the highest probability)
# y_pred = y_pred_proba.argmax(axis=1)

# qwk_score = cohen_kappa_score(y_test, y_pred, weights="quadratic")
# print(f"Quadratic Weighted Kappa: {qwk_score:.4f}")


In [ ]:
# pd.set_option('display.max_rows', 10)
# possible_cat_features = ['FGC-FGC_CU_Zone','FGC-FGC_GSND_Zone','FGC-FGC_GSD_Zone','FGC-FGC_PU_Zone','FGC-FGC_SRL_Zone','FGC-FGC_SRR_Zone','FGC-FGC_TL_Zone','BIA-BIA_Activity_Level_num','BIA-BIA_Frame_num','PCIAT-PCIAT_01','PCIAT-PCIAT_02','PCIAT-PCIAT_03','PCIAT-PCIAT_04','PCIAT-PCIAT_05','PCIAT-PCIAT_06','PCIAT-PCIAT_07','PCIAT-PCIAT_08','PCIAT-PCIAT_09','PCIAT-PCIAT_10','PCIAT-PCIAT_11','PCIAT-PCIAT_12','PCIAT-PCIAT_13','PCIAT-PCIAT_14','PCIAT-PCIAT_15','PCIAT-PCIAT_16','PCIAT-PCIAT_17','PCIAT-PCIAT_18','PCIAT-PCIAT_19','PCIAT-PCIAT_20','PreInt_EduHx-computerinternet_hoursday']
# cat_features = [col for col in X_full_train.columns if col in possible_cat_features]
# catboost = CatBoostClassifier(iterations=1000,
#                     depth=6,
#                     learning_rate=0.05,
#                     loss_function='MultiClass',
#                     verbose=False,
#                     l2_leaf_reg=4,
#                     cat_features=cat_features
#                     )


# # lightgbm = LGBMClassifier(n_estimators=500, learning_rate=0.05, max_depth=6)
# lightgbm = LGBMClassifier(n_estimators=500, learning_rate=0.05, max_depth=6, verbose=-1)

# model = VotingClassifier(estimators=[('catboost', catboost), ('lightgbm', lightgbm)], voting='soft')

# model.fit(X_full_train, y_full_train)

# # Train the model
# model.fit(X_full_train, y_full_train)

# # Make the prediction using the resulting model
# y_pred = model.predict(X_test)

# # Calculate the Quadratic Weighted Kappa score
# qwk_score = cohen_kappa_score(y_test, y_pred, weights="quadratic")
# print(f"Quadratic Weighted Kappa: {qwk_score:.4f}")
# print(f"\nQuadratic Weighted Kappa: {calculate_qwk(y_test, y_pred):.4f}\n")
# print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
# print(classification_report(y_test, y_pred, zero_division=0))

# # ids = X_full_train['id']
# # y_pred.to_csv('training_submission.csv', index=False)


# Apply model to test set

In [ ]:
# ids = test_df['id']
# X_final = test_df.drop(columns=['id'] + [col for col in test_df.columns if 'PCIAT-PCIAT' in col])
# y_pred_final = model.predict(X_final)
# y_pred_final = y_pred_final.argmax(axis=1)

# submission = pd.DataFrame({
#     'id': ids,
#     'prediction': y_pred_final
# })
# submission.to_csv('submission.csv', index=False)
# submission
